<a href="https://colab.research.google.com/github/livig16/llm-foolery/blob/main/Plato_LLM_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Athenian Main Character Syndrome:
### Status 11/30
#### Data:
Initial .jsonl dataset consisting of all statement-response pairings from script-format dialogues is constructed. Possible future dataset constructions include prompt template with speaker as input, so finetuned model could respond as a requested character.
#### Model training:
All working up to last code block, lines 5-6. Unclear issue, perhaps with dataset formatting?

### Install necessary modules

In [ ]:
!pip install jsonlines
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.6 MB/s eta 0:00:00


### Import modules

In [ ]:
import jsonlines  # *=needs install on colab
import re
import torch
#from llama import BasicModelRunner
import datasets #*
from datasets import load_dataset
import tempfile
import logging
import random
import os
import yaml
import time
import transformers #*
import pandas as pd
from sklearn.model_selection import train_test_split

#logger = logging.getLogger(__name__)
#global_config = None

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    AutoModelForCausalLM,
    DataCollatorWithPadding,
    Trainer)


ModuleNotFoundError: ignored

### Format scripts into .jsonl dataset file

In [ ]:
'''
# fn to slice speaker name (if present) off beginning of lines
# and format into speaker/response dict entry
def dict_format(speak:str, resp:str):
    entry = {"speaker": re.sub("^\w+: ", "", speak), "response": re.sub("^\w+: ", "", resp)}
    return entry

In [ ]:
'''
# read in lines of dialogue from dialogue scripts; store as speaker/response examples
dialogues = ["cratylus", "crito", "euthyphro", "gorgias", "greater_hippias", "ion",
              "laches", "lesser_hippias", "meno", "phaedrus", "sophist", "theaetetus"]

with jsonlines.open("plato_dataset.jsonl", mode="w") as write_to:
    for name in dialogues:
        read_from = open(f"/{name}"+"_script.txt", "r", encoding="utf-8")
        lines = read_from.readlines()
        for i in range(len(lines)-1):
            write_to.write(dict_format(lines[i], lines[i+1]))
read_from.close()



# read line, pair with next line as "speaker/response" with "Socrates:"...etc cut out

#jsonlines: refer to https://jsonlines.readthedocs.io/en/latest/



### Set dataset, model, and training config

In [ ]:
dataset_name = "plato_dataset.jsonl"
dataset_path = f"/content/{dataset_name}"
use_hf = False

In [ ]:
model_name = "EleutherAI/pythia-70m"

training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

### Load, tokenize, split dataset

In [ ]:
# Function to tokenize loaded dataset
# & preserve format using .map()
def tokenize_function(examples):
    # extract text
    text = "".join(examples.values())

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
        truncation=True,
        max_length=2048
    )

    return tokenized_inputs

In [ ]:
dataset = load_dataset("json", data_files=dataset_name, split="train")
tokenized_dataset = dataset.map(tokenize_function)
train, test = train_test_split(tokenized_dataset)

### Set training hardware & model parameters

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

device_count = torch.cuda.device_count()
if device_count > 0:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [ ]:
batch_size = 20
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#set training args for Trainer object
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-3,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  #max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=batch_size,

  # Directory to save model checkpoints
  output_dir=f"{model_name}_athenian_mc_syndrome",

  # Other arguments
  #overwrite_output_dir=False, # Overwrite the content of the output directory
  #disable_tqdm=False, # Disable progress bars
  #eval_steps=120, # Number of update steps between two evaluations
  #save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=batch_size, # Batch size for evaluation
  evaluation_strategy="epoch",
  save_strategy="epoch",
  #logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

### Train the damn thing

In [ ]:
# create trainer object
trainer = Trainer(
    model=base_model,
    args=training_args, # hyperparameters
    train_dataset=train, # training data
    eval_dataset=test, # validation data
    tokenizer=tokenizer, # define tokenizer
    data_collator=data_collator, # pads examples in each batch to be equal length
)

# train model
trainer.train()

KeyError: ignored